In [83]:
import csv
import numpy as np
import glob
import os
import sys
import pdb
import os.path as osp
import xml.etree.ElementTree as ETree
sys.path.append(os.getcwd())

import torch
import numpy as np
from stl import mesh
from tqdm import tqdm

from phc.utils.torch_humanoid_batch import Humanoid_Batch
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from collections import defaultdict
from scipy.spatial.transform import Rotation as sRot
import copy

import joblib
from smpl_sim.smpllib.smpl_parser import (
    SMPL_Parser,
    SMPLH_Parser,
    SMPLX_Parser, 
)
from smpl_sim.smpllib.smpl_joint_names import SMPL_MUJOCO_NAMES, SMPL_BONE_ORDER_NAMES, SMPLH_BONE_ORDER_NAMES, SMPLH_MUJOCO_NAMES


smpl_parser_n = SMPL_Parser(model_path="data/smpl", gender="neutral")

with initialize(version_base=None, config_path="phc/data/cfg/"):
    cfg = compose(overrides=["+robot=unitree_g1_29dof_fitting"])
humanoid_fk = Humanoid_Batch(cfg.robot)




2025-03-10 16:43:47,937 - DEBUG - Setting JobRuntime:name=notebook


100%|████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 80248.16it/s]


In [64]:
atlas_base = "phc/data/assets/robot/unitree_g1/"
atlas_path = osp.join(atlas_base, "g1_23dof.xml")
tree = ETree.parse(atlas_path)
xml_doc_root = tree.getroot()
xml_world_body = xml_doc_root.find("worldbody")

xml_assets = xml_doc_root.find("asset")
all_mesh = xml_assets.findall(".//mesh")

geoms = xml_world_body.findall(".//geom")

all_joints = xml_world_body.findall(".//joint")
all_joint_names = [m.attrib['name'] for m in all_joints]
all_motors = tree.findall(".//motor")
all_motors_names = [m.attrib['name'] for m in all_motors]
all_bodies = xml_world_body.findall(".//body")
all_bodies_names = [m.attrib['name'] for m in all_bodies]


print(len(all_bodies_names))
np.array(all_bodies_names)

30


array(['pelvis', 'left_hip_pitch_link', 'left_hip_roll_link',
       'left_hip_yaw_link', 'left_knee_link', 'left_ankle_pitch_link',
       'left_ankle_roll_link', 'right_hip_pitch_link',
       'right_hip_roll_link', 'right_hip_yaw_link', 'right_knee_link',
       'right_ankle_pitch_link', 'right_ankle_roll_link',
       'waist_yaw_link', 'waist_roll_link', 'torso_link',
       'left_shoulder_pitch_link', 'left_shoulder_roll_link',
       'left_shoulder_yaw_link', 'left_elbow_link',
       'left_wrist_roll_link', 'left_wrist_pitch_link',
       'left_wrist_yaw_link', 'right_shoulder_pitch_link',
       'right_shoulder_roll_link', 'right_shoulder_yaw_link',
       'right_elbow_link', 'right_wrist_roll_link',
       'right_wrist_pitch_link', 'right_wrist_yaw_link'], dtype='<U25')

In [54]:
dof_subset = []
for i, k in enumerate(all_bodies_names[1:]):
    if "wrist" in k:
        continue
    dof_subset.append(i)
dof_subset = np.array(dof_subset)

In [106]:
full_lafan = {}
fps = 30
seq_name = "dance1_subject1"
for file_name in os.listdir("../../nv/LAFAN1_Retargeting_Dataset/g1/"):
    joint_data = []
    seq_name = file_name.split(".")[0]
    print(seq_name)
    with open(f'../../nv/LAFAN1_Retargeting_Dataset/g1/{seq_name}.csv', mode='r', newline='') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            joint_data.append([float(i) for i in row])
    joint_data = np.array(joint_data)
    
    
    dof = joint_data[:, 7:][:, dof_subset]
    root_rot = joint_data[:, 3:7]
    root_trans_offset = joint_data[:, :3]
    
    N = dof.shape[0]
    dof_nums = np.arange(humanoid_fk.num_dof).tolist()
    dof_pos_selected = (humanoid_fk.dof_axis[None, ] * dof[..., None])[None, ]
    pose_aa_h1_new = np.zeros((1, N, len(humanoid_fk.body_names), 3))
    pose_aa_h1_new[:, :, humanoid_fk.actuated_joints_idx[np.array(dof_nums) + 1], :] = dof_pos_selected[:, :, dof_nums]
    pose_aa_h1_new[:, :, 0:1, :] = sRot.from_quat(root_rot).as_rotvec()[None, :, None, :]
    pose_aa_h1_new = pose_aa_h1_new[0]
    
    
    dump = {
        "root_trans_offset": root_trans_offset.astype(np.float32),
        "pose_aa": pose_aa_h1_new.astype(np.float32),
        "dof": dof.astype(np.float32), 
        "root_rot": root_rot.astype(np.float32),
        "smpl_joints": np.zeros((root_trans_offset.shape[0], 24, 3)).astype(np.float32), 
        "fps": fps
    }
    dump_dict = {seq_name: dump}
    full_lafan[seq_name] = dump 
    joblib.dump(dump_dict, f"data/g1/v1/singles/{seq_name}.pkl", compress=True)
    
joblib.dump(full_lafan, f"data/g1/v1/lafan.pkl")

fallAndGetUp1_subject4
dance1_subject3
dance2_subject4
fightAndSports1_subject1
jumps1_subject1
fight1_subject3
run1_subject5
dance1_subject2
walk1_subject5
run2_subject1
sprint1_subject4
jumps1_subject5
sprint1_subject2
walk2_subject1
walk3_subject4
walk3_subject2
fallAndGetUp2_subject3
dance1_subject1
fight1_subject5
fallAndGetUp1_subject5
fallAndGetUp2_subject2
walk3_subject3
jumps1_subject2
walk2_subject4
run1_subject2
walk1_subject1
run2_subject4
walk3_subject1
dance2_subject5
dance2_subject1
walk4_subject1
dance2_subject2
walk2_subject3
fallAndGetUp1_subject1
walk1_subject2
fallAndGetUp3_subject1
dance2_subject3
fight1_subject2
walk3_subject5
fightAndSports1_subject4


['data/g1/v1/lafan.pkl']

In [107]:
subset_key = 'walk1_subject5'
start, end = 90, 120
data_dump = {}
for k, v in full_lafan[subset_key].items():
    if not k in ['fps']:
        data_dump[k] = v[start:end]
    else:
        data_dump[k] = v
joblib.dump({subset_key: data_dump}, "data/g1/v1/singles/walk_short.pkl", compress=True)


['data/g1/v1/singles/walk_short.pkl']

In [73]:
data_dump['dof'][0]

array([-0.151367, -0.118991,  0.05803 ,  0.453353, -0.461074,  0.058986,
       -0.600443, -0.166162, -0.128207,  0.999351, -0.336191, -0.067373,
       -0.012413, -0.056357,  0.026593,  0.004278,  0.177925,  0.134414,
        1.156344,  0.235976, -0.232575, -0.0441  ,  0.740262],
      dtype=float32)

In [75]:
data_dump['pose_aa'].shape

(30, 30, 3)